In [3]:
"""from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableMap

chat = ChatOpenAI(temperature=0.1)

prouduct_prompt = ChatPromptTemplate.from_messages([
  ("system", "you are a product manager. you will help to build a ai product.if you are not sure about something say 'i am not sure about that'"),
  ("human", "{user_goal_prompt}"),
])
prouduct_chain = prouduct_prompt | chat

In [ ]:
"""
design_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a product designer. You will help design the AI product. If you're not sure about something, say 'I'm not sure about that'."),
    ("human", "{product}"),
])
design_chain = design_prompt | chat

In [ ]:
"""
devolper_prompt = ChatPromptTemplate.from_messages([
  ("system","You are a developer. You will write code for the AI product based on product and design. If you're not sure about something, say 'I'm not sure about that'."),
  ("human","Product Info:\n{product}\n\nDesign Info:\n{design}"),
]
)
devolper_chain = devolper_prompt| chat

In [ ]:
"""
def full_chain(user_goal_prompt: str):
  product = prouduct_chain.invoke({"user_goal_prompt": user_goal_prompt}).content
  design = design_chain.invoke({"product": product}).content
  code = devolper_chain.invoke({"product": product, "design": design}).content
  return {
      "product": product,
      "design": design,
      "code": code
  }

In [ ]:
"""
result = full_chain("I want to build a decision-making AI agent that helps our sales team determine whether a merchant is eligible for a fee discount.The agent should read our internal policy documents and, when the sales team types in a merchant name and case, the agent should return a yes/no answer with explanation.The goal is to avoid manual back-and-forth between sales and operations.")
print("🔷 Product:\n", result["product"])
print("\n🎨 Design:\n", result["design"])
print("\n💻 Code:\n", result["code"])
"""

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)


example_prompt = ChatPromptTemplate.from_messages([
  ("human", "{question}"),
  ("ai", "{answer}"),
])

example = [
  {"question": "what is football?", "answer": "Football is a team sport played between two teams of eleven players with a spherical ball."},
  {"question": "what is basketball?", "answer": "Basketball is a team sport in which two teams, most commonly of five players each, opposing one another on a rectangular court."},
]                                             

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt = example_prompt,
  examples = example,
  suffix= "Human:What do you know about {Sport}?",
  input_variables=["Sport"]
)

formatted = prompt.format(Sport="f1")
response = chat.invoke(formatted)

prompt.format(Sport="f1")

JSONDecodeError: Illegal trailing comma before end of object: line 9 column 4 (char 339)

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = OpenAI(
  temperature=0.1,
  max_tokens=500,
  model = "gpt-3.5-turbo-16k",)

chat.save("model.json")

/var/folders/0n/m31n514n1vg206kjl12c6w1m0000gn/T/ipykernel_13430/2066542689.py:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  chat = OpenAI(


In [15]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "hi"}, {"output": "how are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='how are you?', additional_kwargs={}, response_metadata={})]}

In [18]:
memory.save_context({"input": "hi"}, {"output": "how are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='how are you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='how are you?', additional_kwargs={}, response_metadata={})]}

In [19]:
memory.save_context({"input": "hi"}, {"output": "how are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='how are you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='how are you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='how are you?', additional_kwargs={}, response_metadata={})]}

In [21]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("hi i'm hyuk, i live in pangyo", "wow that's great")

In [22]:
add_message("South korea is best place ever", "i agree with you")

In [23]:
get_history()

{'history': 'Hyuk introduces themselves as living in Pangyo, South Korea. The AI responds positively, agreeing that South Korea is the best place ever.'}